In [1]:
def homework(train_X, train_y, test_X):
    # ここにKerasのコードを書く. 
    import keras
    from keras.models import Sequential
    from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, BatchNormalization
    from keras.layers import advanced_activations
    from keras.layers.core import Dropout
    from keras import backend as K
    
    
    #
    from keras.callbacks import LearningRateScheduler
    from keras import optimizers
    start = 0.002
    stop = 0.001
    nb_epoch = 1000
    learning_rates = np.linspace(start, stop, nb_epoch)
    #
    
    def for_flat(model):
        act = keras.layers.advanced_activations.PReLU()
        model.add(Dense(120, kernel_initializer='he_normal'))
        model.add(act)
        return model
    
    def swish(x):
        return x *K.sigmoid(x)
        
    model = Sequential()
     # 入力画像 32x32x3  (縦の画素数)x(横の画素数)x(チャンネル数)
     # 32x32x3 -> 28x28x8
    model.add(Conv2D(8, kernel_size=(5, 5), activation=swish,input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    # 28x28x8 -> 24x24x16
    model.add(Conv2D(16, kernel_size=(5, 5), activation=swish))
    # 24x24x16 -> 12x12x16 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # 12x12x16 -> 8x8x32
    model.add(Conv2D(32, kernel_size=(5, 5), activation=swish))
    # 4x4x32
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    model.add(Dropout(0.25))
    # 5x5x8 -> 200 
    model.add(Flatten()) 
    model.add(Dense(256, activation=swish, kernel_initializer='he_normal'))
    model.add(Dense(128, activation=swish, kernel_initializer='he_normal'))
    model.add(Dense(64, activation=swish, kernel_initializer='he_normal'))
#    model = for_flat(model)
    model.add(Dense(32, activation=swish, kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))  # 84 ->10
    change_lr = LearningRateScheduler(lambda epoch: float(learning_rates[epoch]))#
    sgd = keras.optimizers.SGD(lr=start, momentum=0.9, nesterov=True)#
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam')
    
    model.fit(x=train_X, y=train_y, batch_size=32, epochs=8, validation_split=0.1,callbacks=[change_lr])
    
    pred_y = model.predict(test_X)
    pred_y = np.argmax(pred_y, 1)
    
    return pred_y

In [2]:
import numpy as np

from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

random_state = 42

def load_cifar():
    (cifar_X_1, cifar_y_1), (cifar_X_2, cifar_y_2) = cifar10.load_data()

    cifar_X = np.r_[cifar_X_1, cifar_X_2]
    cifar_y = np.r_[cifar_y_1, cifar_y_2]

    cifar_X = cifar_X.astype('float32') / 255
    cifar_y = np.eye(10)[cifar_y.astype('int32').flatten()]

    train_X, test_X, train_y, test_y = train_test_split(cifar_X, cifar_y,
                                                        test_size=10000,
                                                        random_state=42)

    return (train_X, test_X, train_y, test_y)

def score_homework():
    global test_X, test_y, pred_y
    train_X, test_X, train_y, test_y = load_cifar()
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(np.argmax(test_y, 1), pred_y, average='macro'))

if __name__ == '__main__':
    score_homework()

/home/jmbo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 45000 samples, validate on 5000 samples
Epoch 1/8
45000/45000 [==============================] - 311s 7ms/step - loss: 1.8246 - val_loss: 1.4972
Epoch 2/8
45000/45000 [==============================] - 311s 7ms/step - loss: 1.5511 - val_loss: 1.5368
Epoch 3/8
45000/45000 [==============================] - 311s 7ms/step - loss: 1.4432 - val_loss: 1.2955
Epoch 4/8
45000/45000 [==============================] - 307s 7ms/step - loss: 1.3657 - val_loss: 1.3021
Epoch 5/8
45000/45000 [==============================] - 299s 7ms/step - loss: 1.3121 - val_loss: 1.2635
Epoch 6/8
45000/45000 [==============================] - 306s 7ms/step - loss: 1.2699 - val_loss: 1.3728
Epoch 7/8
45000/45000 [==============================] - 321s 7ms/step - loss: 1.2395 - val_loss: 1.1747
Epoch 8/8
45000/45000 [==============================] - 378s 8ms/step - loss: 1.2219 - val_loss: 1.1315
0.6180736666579502
